In [119]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import nltk
nltk.download('vader_lexicon')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

[nltk_data] Error loading vader_lexicon: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
['Womens Clothing E-Commerce Reviews.csv']


In [120]:
#Load all data
df= pd.read_csv("../input/Womens Clothing E-Commerce Reviews.csv" )
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [121]:
#Exclusion Word & Symbol List  
stopwords = nltk.corpus.stopwords.words('english')
fdist1 = None
symbols = ["'", '"', '`', '.', ',', '-', '!', '?', ':', ';', '(', ')','...',"''",'&','``','--','..']
exclude = ["'s","'m","small\\r\\","\\r\\n\\r\\"]

In [122]:
#Grand total % proportion cumulation score 
#Example:
#(key word counts per single "Review Text" entry / all keyword counts) = Total % Proportion
tokens = nltk.word_tokenize(df['Review Text'].to_string())
fdist1 = nltk.FreqDist(w.lower() for w in tokens if w.lower() not in stopwords + symbols + exclude and len(w) > 1 and w.isnumeric() == False)

fdist1_df =  pd.DataFrame(fdist1.most_common(), columns = ('Item','Count'))
fdist1_df['Prop'] = fdist1_df['Count']/sum(fdist1_df['Count']) 

df["GrandTotalProp"] = 0
df['ContainWord'] = None
try:
    for index, row in fdist1_df.iterrows(): 
        #print(df["Review Text"].str.contains(row[0]))
        df['ContainWord'] = df["Review Text"].str.contains(row[0])

        df.loc[df['ContainWord'] == True, 'GrandTotalProp'] = df.loc[df['ContainWord'] == True, 'GrandTotalProp'] + row[2]

        #if index > 100: break
        if row[1]  < 3: break
except Exception as e:         
    print(row[0])
    print(row[1])
    raise
    
df.head()    

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,GrandTotalProp,ContainWord
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates,0.013793,False
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses,0.172093,False
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,0.178479,False
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,0.090387,False
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,0.098637,False


In [123]:
ClassName = df['Class Name'].unique()
#ClassName


In [124]:
#Total % Proportion by 'Class Name'
df["TotalProp"] = 0
df['ContainWord'] = None
for c in ClassName: 
    tokens = nltk.word_tokenize(df[df['Class Name']== c]['Review Text'].to_string())
    fdist1 = nltk.FreqDist(w.lower() for w in tokens if w.lower() not in stopwords + symbols + exclude and len(w) > 1 and w.isnumeric() == False)
    tmp =  pd.DataFrame(fdist1.most_common(), columns = ('Item','Count'))
    tmp['Prop'] = tmp['Count']/sum(tmp['Count'])
    
    for index, row in tmp.iterrows(): 
        df.loc[df['Class Name']== c,'ContainWord'] = df[df['Class Name']== c]["Review Text"].str.contains(row[0])        
        df.loc[(df['Class Name']== c) & (df['ContainWord'] == True), 'TotalProp'] = df.loc[(df['Class Name']== c) & (df['ContainWord'] == True), 'TotalProp'] + row[2]
        
        #if index > 10: break
        if row[1] < 3: break    
            
df.head()             


,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,GrandTotalProp,ContainWord,TotalProp
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates,0.013793,False,0.022659
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses,0.172093,False,0.272147
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,0.178479,False,0.252226
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,0.090387,True,0.107672
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,0.098637,True,0.105174


In [125]:
#'compound', 'neg', 'neu', 'pos' score by vader_analyzer
vader_analyzer = SentimentIntensityAnalyzer()
try:
    df = df.join(pd.DataFrame([vader_analyzer.polarity_scores(str(txt)) for txt in df["Review Text"]  ]))
except Exception as e:         
    print(txt)
    raise
    
df.head()    

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,GrandTotalProp,ContainWord,TotalProp,compound,neg,neu,pos
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates,0.013793,False,0.022659,0.8932,0.000,0.272,0.728
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses,0.172093,False,0.272147,0.9729,0.000,0.664,0.336
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,0.178479,False,0.252226,0.9427,0.027,0.792,0.181
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,0.090387,True,0.107672,0.5727,0.226,0.340,0.434
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,0.098637,True,0.105174,0.9291,0.000,0.700,0.300


In [126]:
#Newly Generated Features  
df[['GrandTotalProp','TotalProp','compound', 'neg', 'neu', 'pos']].head()

,GrandTotalProp,TotalProp,compound,neg,neu,pos
0,0.013793,0.022659,0.8932,0.000,0.272,0.728
1,0.172093,0.272147,0.9729,0.000,0.664,0.336
2,0.178479,0.252226,0.9427,0.027,0.792,0.181
3,0.090387,0.107672,0.5727,0.226,0.340,0.434
4,0.098637,0.105174,0.9291,0.000,0.700,0.300
